## Imports

In [ ]:
import datetime
import gc

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
cuda_use_gpus(1)

In [ ]:
from keras import backend as K
from keras.models import Model
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint

## Config

In [ ]:
feature_list_id = 'oofp_lystdo_bi_lstm'

In [ ]:
RANDOM_SEED = 42

In [ ]:
np.random.seed(RANDOM_SEED)

## Read Data

In [ ]:
embedding_matrix = load(aux_data_folder + 'embedding_weights_fasttext_filtered_no_stopwords.pickle')

In [ ]:
X_train_q1 = load(features_data_folder + 'X_train_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_train_q2 = load(features_data_folder + 'X_train_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [ ]:
y_train = load(features_data_folder + 'y_train.pickle')

In [ ]:
EMBEDDING_DIM = embedding_matrix.shape[-1]
VOCAB_LENGTH = embedding_matrix.shape[0]
MAX_SEQUENCE_LENGTH = X_train_q1.shape[-1]

In [ ]:
print(EMBEDDING_DIM, VOCAB_LENGTH, MAX_SEQUENCE_LENGTH)

## Train Models & Compute Out-of-Fold Predictions

In [ ]:
def create_model():
    params = {
        'dense_dropout_rate': 0.07512852175097123,
        'lstm_dropout_rate': 0.3320541002991107,
        'num_dense': 130,
        'num_lstm': 333
    }
    
    embedding_layer = Embedding(
        VOCAB_LENGTH,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,
    )
    lstm_layer = Bidirectional(LSTM(
        params['num_lstm'],
        dropout=params['lstm_dropout_rate'],
        recurrent_dropout=params['lstm_dropout_rate'],
    ))

    sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences_1 = embedding_layer(sequence_1_input)
    x1 = lstm_layer(embedded_sequences_1)

    sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences_2 = embedding_layer(sequence_2_input)
    y1 = lstm_layer(embedded_sequences_2)

    merged = concatenate([x1, y1])
    merged = Dropout(params['dense_dropout_rate'])(merged)
    merged = BatchNormalization()(merged)

    merged = Dense(params['num_dense'], activation='relu')(merged)
    merged = Dropout(params['dense_dropout_rate'])(merged)
    merged = BatchNormalization()(merged)

    output = Dense(1, activation='sigmoid')(merged)

    model = Model(
        inputs=[sequence_1_input, sequence_2_input],
        outputs=output
    )

    model.compile(
        loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['accuracy']
    )

    return model

In [ ]:
model_checkpoint_path = aux_data_folder + 'fold-checkpoint-' + feature_list_id + '.h5'

In [ ]:
NUM_FOLDS = 5
NUM_EPOCHS = 35

In [ ]:
kfold = StratifiedKFold(
    n_splits=NUM_FOLDS,
    shuffle=True,
    random_state=RANDOM_SEED
)

In [ ]:
y_train_oofp = np.zeros_like(y_train, dtype='float64')

In [ ]:
%%time

for fold_num, (ix_train, ix_val) in enumerate(kfold.split(X_train_q1, y_train)):
    X_fold_train_q1 = np.vstack([X_train_q1[ix_train], X_train_q2[ix_train]])
    X_fold_train_q2 = np.vstack([X_train_q2[ix_train], X_train_q1[ix_train]])

    X_fold_val_q1 = np.vstack([X_train_q1[ix_val], X_train_q2[ix_val]])
    X_fold_val_q2 = np.vstack([X_train_q2[ix_val], X_train_q1[ix_val]])

    y_fold_train = np.concatenate([y_train[ix_train], y_train[ix_train]])
    y_fold_val = np.concatenate([y_train[ix_val], y_train[ix_val]])
    
    print()
    print(f'Fitting fold {fold_num + 1} of {kfold.n_splits}')
    print()
    
    model = create_model()
    model.fit(
        [X_fold_train_q1, X_fold_train_q2], y_fold_train,
        validation_data=([X_fold_val_q1, X_fold_val_q2], y_fold_val),
        class_weight=keras_get_class_weights(y_fold_val),

        batch_size=384,
        epochs=NUM_EPOCHS,
        verbose=1,

        callbacks=[
            EarlyStopping(
                monitor='val_loss',
                min_delta=0.001,
                patience=3,
                verbose=1,
                mode='auto',
            ),
            ModelCheckpoint(
                model_checkpoint_path,
                monitor='val_loss',
                save_best_only=True,
                verbose=2,
            ),
        ],
    )
        
    # Create out-of-fold prediction.
    model.load_weights(model_checkpoint_path)
    
    y_pred_oofp = model.predict([X_train_q1[ix_val], X_train_q2[ix_val]], batch_size=2048, verbose=1).reshape(-1)
    y_pred_oofp += model.predict([X_train_q2[ix_val], X_train_q1[ix_val]], batch_size=2048, verbose=1).reshape(-1)
    y_pred_oofp /= 2
       
    # Remember them.
    y_train_oofp[ix_val] = y_pred_oofp
    
    K.clear_session()
    del X_fold_train_q1
    del X_fold_train_q2
    del X_fold_val_q1
    del X_fold_val_q2
    del model
    gc.collect()

## Save feature names

In [ ]:
feature_names = [
    'oofp_lystdo_bi_lstm',
]

In [ ]:
save_lines(feature_names, features_data_folder + f'X_train_{feature_list_id}.names')

## Save Train features

In [ ]:
y_train_oofp = y_train_oofp.reshape((-1, 1))

In [ ]:
save(y_train_oofp, features_data_folder + f'X_train_{feature_list_id}.pickle')

## Save Test features

In [ ]:
X_test_q1 = load(features_data_folder + 'X_test_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_test_q2 = load(features_data_folder + 'X_test_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [ ]:
model = create_model()
model.load_weights(model_checkpoint_path)

In [ ]:
# It would be better to fit the model on the whole training set
# but the validation set for early stopping would be an issue.
y_test_oofp = model.predict([X_test_q1, X_test_q2], batch_size=2048, verbose=1).reshape(-1)
y_test_oofp += model.predict([X_test_q2, X_test_q1], batch_size=2048, verbose=1).reshape(-1)
y_test_oofp /= 2

In [ ]:
y_test_oofp = y_test_oofp.reshape((-1, 1))

In [ ]:
save(y_test_oofp, features_data_folder + f'X_test_{feature_list_id}.pickle')